<a href="https://colab.research.google.com/github/Ananya221203/kaggle-compitition-/blob/main/6_comp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'tss24-competition-4:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F82149%2F8952049%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240629%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240629T125826Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D31a4b074743157ca8728dcfe88b01214bdbd01750e00811aef693d8a6946be665e97a0bcc4fbf7ef1687c73d99e159b8d2252e584b07a8a4ffe598fd1c12880d93686d0b22f032bd72fc3349c915ae8585fe8ecc824bc018f06115c6d4c1c873eebbfc0467b07da5e1a844ea4da3c60a079fec5bbaac1985e0402e29170e924b800e7f1d1b5cf17319aa6d2720f2d35680d005ac1e2aa0f7e0d51959c79dd9cbca6823323e0e9037211339b5f0cde0e8c30c472552d8a091e183801b529f616d741d77c51ca4dbe152db7ebc124cd3d00006c11550304f8b7443901101806de7e3f9e0225682abc14e3eda7f11afe9496518bfb7e58be0ce185b6684dfe4f664'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 2529839 bytes downloaded
Downloaded and uncompressed: tss24-competition-4
Data source import complete.


In [3]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [4]:
test = pd.read_csv("/kaggle/input/tss24-competition-4/test.csv", index_col = "id")
test = pd.get_dummies(test, dtype=float)
test

,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Sex_F,Sex_I,Sex_M
id,,,,,,,,,,
90615,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005,0.0,0.0,1.0
90616,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750,0.0,0.0,1.0
90617,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405,0.0,0.0,1.0
90618,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350,0.0,0.0,1.0
90619,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
151021,0.345,0.260,0.085,0.1775,0.0735,0.0265,0.0500,0.0,1.0,0.0
151022,0.525,0.410,0.145,0.8445,0.3885,0.1670,0.2050,1.0,0.0,0.0
151023,0.590,0.440,0.155,1.1220,0.3930,0.2000,0.2650,0.0,1.0,0.0


In [5]:
train = pd.read_csv("/kaggle/input/tss24-competition-4/train.csv", index_col = "id")
train = pd.get_dummies(train, dtype=float)
train

,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings,Sex_F,Sex_I,Sex_M
id,,,,,,,,,,,
0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11,1.0,0.0,0.0
1,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11,1.0,0.0,0.0
2,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6,0.0,1.0,0.0
3,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10,0.0,0.0,1.0
4,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
90610,0.335,0.235,0.075,0.1585,0.0685,0.0370,0.0450,6,0.0,0.0,1.0
90611,0.555,0.425,0.150,0.8790,0.3865,0.1815,0.2400,9,0.0,0.0,1.0
90612,0.435,0.330,0.095,0.3215,0.1510,0.0785,0.0815,6,0.0,1.0,0.0


In [6]:
corr_matrix = train.corr()
corr_matrix

,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings,Sex_F,Sex_I,Sex_M
Length,1.000000,0.989732,0.916094,0.931449,0.909609,0.913134,0.911073,0.623786,0.365578,-0.656415,0.315624
Diameter,0.989732,1.000000,0.919618,0.933848,0.908466,0.914668,0.917871,0.636832,0.370765,-0.664771,0.319131
Height,0.916094,0.919618,1.000000,0.902344,0.861769,0.886132,0.904019,0.665772,0.367521,-0.645291,0.302475
Whole weight,0.931449,0.933848,0.902344,1.000000,0.971249,0.974319,0.964201,0.617274,0.368940,-0.651185,0.307096
Whole weight.1,0.909609,0.908466,0.861769,0.971249,1.000000,0.949227,0.911800,0.515067,0.343068,-0.619272,0.299513
Whole weight.2,0.913134,0.914668,0.886132,0.974319,0.949227,1.000000,0.937069,0.588954,0.370807,-0.646484,0.300536
Shell weight,0.911073,0.917871,0.904019,0.964201,0.911800,0.937069,1.000000,0.694766,0.372127,-0.648101,0.300911
Rings,0.623786,0.636832,0.665772,0.617274,0.515067,0.588954,0.694766,1.000000,0.296411,-0.516591,0.240048
Sex_F,0.365578,0.370765,0.367521,0.368940,0.343068,0.370807,0.372127,0.296411,1.000000,-0.487569,-0.463848
Sex_I,-0.656415,-0.664771,-0.645291,-0.651185,-0.619272,-0.646484,-0.648101,-0.516591,-0.487569,1.000000,-0.547320


In [7]:
train.head()

,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings,Sex_F,Sex_I,Sex_M
id,,,,,,,,,,,
0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11,1.0,0.0,0.0
1,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11,1.0,0.0,0.0
2,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6,0.0,1.0,0.0
3,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10,0.0,0.0,1.0
4,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9,0.0,1.0,0.0


In [8]:
train.describe()

,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings,Sex_F,Sex_I,Sex_M
count,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000
mean,0.517098,0.401679,0.135464,0.789035,0.340778,0.169422,0.225898,9.696794,0.292391,0.365204,0.342405
std,0.118217,0.098026,0.038008,0.457671,0.204428,0.100909,0.130203,3.176221,0.454863,0.481490,0.474517
min,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000,0.000000,0.000000,0.000000
25%,0.445000,0.345000,0.110000,0.419000,0.177500,0.086500,0.120000,8.000000,0.000000,0.000000,0.000000
50%,0.545000,0.425000,0.140000,0.799500,0.330000,0.166000,0.225000,9.000000,0.000000,0.000000,0.000000
75%,0.600000,0.470000,0.160000,1.067500,0.463000,0.232500,0.305000,11.000000,1.000000,1.000000,1.000000
max,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000,1.000000,1.000000,1.000000


In [9]:
duplicate=train.duplicated()
len(duplicate)

90615

In [10]:
X = train.drop(['Rings'], axis=1)
y = train['Rings']

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

X_val_scaled = scaler.transform(X_val)


In [13]:
from sklearn.ensemble import GradientBoostingRegressor


model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,  random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_val)



In [14]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"Mean Squared Error: {mse}")
print("R-squared:", r2)


Mean Squared Error: 3.6820789017848137
R-squared: 0.6415545843816346


In [15]:
test = pd.read_csv("/kaggle/input/tss24-competition-4/test.csv", index_col="id")
test = pd.get_dummies(test, dtype=float)
test

,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Sex_F,Sex_I,Sex_M
id,,,,,,,,,,
90615,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005,0.0,0.0,1.0
90616,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750,0.0,0.0,1.0
90617,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405,0.0,0.0,1.0
90618,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350,0.0,0.0,1.0
90619,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
151021,0.345,0.260,0.085,0.1775,0.0735,0.0265,0.0500,0.0,1.0,0.0
151022,0.525,0.410,0.145,0.8445,0.3885,0.1670,0.2050,1.0,0.0,0.0
151023,0.590,0.440,0.155,1.1220,0.3930,0.2000,0.2650,0.0,1.0,0.0


In [16]:
test_pred = model.predict(test)
test_pred[test_pred<0] = 1
submission = pd.DataFrame({"Rings": test_pred}, index=test.index)
submission



submission['id'] = submission.index

# Reorder columns if necessary
submission = submission[['id', 'Rings']]
submission


,id,Rings
id,,
90615,90615,9.412111
90616,90616,9.869216
90617,90617,10.417670
90618,90618,10.379386
90619,90619,7.536758
...,...,...
151021,151021,6.420654
151022,151022,9.416519
151023,151023,11.356932


In [17]:
submission.to_csv("submission.csv", index=False)
#submission = pd.DataFrame({'id': test['id'], 'Rings': test_pred})
#submission.to_csv('submission.csv', index=False)
#print("Submission file created successfully.")